In [1]:
# импортируем библиотеки pandas и numpy,
# и классы MissingIndicator и SimpleImputer
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.impute import MissingIndicator, SimpleImputer

In [2]:
# записываем CSV-файл в объект DataFrame
data = pd.read_csv('Data/Verizon_missing.csv', sep=';')
# выведем первые 3 наблюдения
data.head(3)

,longdist,internat,local,age,income,billtype,pay,churn
0,16.0,0.0,5.0,46.0,34805.5,Бюджетный,CC,0
1,0.0,0.0,5.0,59.0,60111.8,NaN,NaN,1
2,13.0,0.0,NaN,NaN,13126.9,Бюджетный,Auto,0


In [3]:
# разбиваем данные на обучающие и тестовые: получаем обучающий
# массив признаков, тестовый массив признаков, обучающий массив
# меток, тестовый массив меток
train, test, y_train, y_test = train_test_split(
    data.drop('churn', axis=1), 
    data['churn'], 
    test_size=.3, 
    stratify=data['churn'], 
    random_state=100)

In [4]:
# взглянем на обучающий массив признаков
train.head(10)

,longdist,internat,local,age,income,billtype,pay
18,19.0,NaN,11.0,88.0,66906.60,Бесплатный,CH
19,19.0,NaN,96.0,79.0,37571.10,Бесплатный,CC
66,0.0,0.0,1.0,94.0,13946.80,Бюджетный,NaN
65,9.0,6.0,12.0,93.0,46771.20,Бюджетный,NaN
74,26.0,0.0,NaN,32.0,NaN,Бюджетный,Auto
96,9.0,0.0,NaN,25.0,52366.40,Бюджетный,CC
47,14.0,0.0,48.0,58.0,4123.46,Бесплатный,CC
107,0.0,0.0,3.0,68.0,14955.80,Бесплатный,CH
11,1.0,0.0,9.0,44.0,11861.40,Бюджетный,CD
86,NaN,4.0,13.0,54.0,27376.00,NaN,CC


In [5]:
# взглянем на тестовый массив признаков
test.head(10)

,longdist,internat,local,age,income,billtype,pay
44,3.0,0.0,9.0,45.0,70239.8,Бесплатный,CC
59,21.0,NaN,NaN,33.0,60170.1,NaN,CH
51,29.0,NaN,110.0,29.0,18861.6,Бюджетный,CC
35,12.0,0.0,55.0,62.0,13965.4,Бесплатный,CH
55,25.0,0.0,77.0,95.0,NaN,NaN,Auto
121,14.0,NaN,167.0,83.0,NaN,Бесплатный,CC
91,19.0,NaN,53.0,30.0,17866.9,Бюджетный,Auto
30,20.0,0.0,29.0,58.0,91446.4,Бюджетный,CC
23,2.0,0.0,99.0,29.0,49127.2,Бесплатный,CC
53,0.0,0.0,1.0,75.0,NaN,Бесплатный,CC


In [6]:
# выясняем, есть ли пропуски
print(train.isnull().sum())
print('')
print(test.isnull().sum())

longdist     5
internat    23
local       12
age          8
income      18
billtype    13
pay         13
dtype: int64

longdist     3
internat    11
local        2
age          0
income      11
billtype    11
pay          2
dtype: int64


In [7]:
# создаем экземпляр класса MissingIndicator
miss_ind = MissingIndicator()
# обучаем модель импутации - создаем индикатор пропусков
# переменной local в обучающем массиве
miss_ind.fit(train[['local']])
# применяем модель импутации к переменной local 
# в обучающем массиве признаков:
# создаем индикатор пропусков переменной local 
# в обучающем массиве
train['miss_ind_local'] = miss_ind.transform(train[['local']])
# применяем модель импутации к переменной local 
# в тестовом массиве признаков:
# создаем индикатор пропусков переменной local 
# в тестовом массиве
test['miss_ind_local'] = miss_ind.transform(test[['local']])
train.head()

,longdist,internat,local,age,income,billtype,pay,miss_ind_local
18,19.0,NaN,11.0,88.0,66906.6,Бесплатный,CH,False
19,19.0,NaN,96.0,79.0,37571.1,Бесплатный,CC,False
66,0.0,0.0,1.0,94.0,13946.8,Бюджетный,NaN,False
65,9.0,6.0,12.0,93.0,46771.2,Бюджетный,NaN,False
74,26.0,0.0,NaN,32.0,NaN,Бюджетный,Auto,True


In [8]:
# взглянем на медиану переменной local
# в обучающем массиве признаков
train['local'].median()

32.5

In [9]:
# создаем экземпляр класса SimpleImputer
simp = SimpleImputer(strategy='median')
# обучаем модель импутации - вычисляем медиану
# переменной local в обучающем массиве
simp.fit(train[['local']]);

In [10]:
# применяем модель импутации к переменной local 
# в обучающем массиве признаков: 
# заменяем пропуски переменной в обучающем массиве признаков 
# медианой переменной, вычисленной по ОБУЧАЮЩЕМУ массиву признаков
train['local'] = simp.transform(train[['local']])
# еще можно так
# train['local'] = simp.transform(train['local'].values.reshape(-1, 1))

# применяем модель импутации к переменной local 
# в тестовом массиве признаков: 
# заменяем пропуски переменной в тестовом массиве признаков 
# медианой переменной, вычисленной по ОБУЧАЮЩЕМУ массиву признаков
test['local'] = simp.transform(test[['local']])

In [11]:
# взглянем на обучающий массив признаков
train.head(10)

,longdist,internat,local,age,income,billtype,pay,miss_ind_local
18,19.0,NaN,11.0,88.0,66906.60,Бесплатный,CH,False
19,19.0,NaN,96.0,79.0,37571.10,Бесплатный,CC,False
66,0.0,0.0,1.0,94.0,13946.80,Бюджетный,NaN,False
65,9.0,6.0,12.0,93.0,46771.20,Бюджетный,NaN,False
74,26.0,0.0,32.5,32.0,NaN,Бюджетный,Auto,True
96,9.0,0.0,32.5,25.0,52366.40,Бюджетный,CC,True
47,14.0,0.0,48.0,58.0,4123.46,Бесплатный,CC,False
107,0.0,0.0,3.0,68.0,14955.80,Бесплатный,CH,False
11,1.0,0.0,9.0,44.0,11861.40,Бюджетный,CD,False
86,NaN,4.0,13.0,54.0,27376.00,NaN,CC,False


In [12]:
# взглянем на обучающий массив признаков
test.head(10)

,longdist,internat,local,age,income,billtype,pay,miss_ind_local
44,3.0,0.0,9.0,45.0,70239.8,Бесплатный,CC,False
59,21.0,NaN,32.5,33.0,60170.1,NaN,CH,True
51,29.0,NaN,110.0,29.0,18861.6,Бюджетный,CC,False
35,12.0,0.0,55.0,62.0,13965.4,Бесплатный,CH,False
55,25.0,0.0,77.0,95.0,NaN,NaN,Auto,False
121,14.0,NaN,167.0,83.0,NaN,Бесплатный,CC,False
91,19.0,NaN,53.0,30.0,17866.9,Бюджетный,Auto,False
30,20.0,0.0,29.0,58.0,91446.4,Бюджетный,CC,False
23,2.0,0.0,99.0,29.0,49127.2,Бесплатный,CC,False
53,0.0,0.0,1.0,75.0,NaN,Бесплатный,CC,False


In [13]:
# создаем список количественных переменных
numeric_cols = ['longdist', 'internat', 'age', 'income']
# обучаем модель импутации - вычисляем медианы переменных
# longdist, internat, age и income в обучающем массиве признаков
simp.fit(train[numeric_cols])
# применяем модель импутации к указанным переменным 
# в обучающем массиве признаков: 
# заменяем пропуски каждой переменной в обучающем
# массиве признаков медианой соответствующей переменной, 
# вычисленной по ОБУЧАЮЩЕМУ массиву признаков
train[numeric_cols] = simp.transform(train[numeric_cols])
# применяем модель импутации к указанным переменным 
# в тестовом массиве признаков: 
# заменяем пропуски каждой переменной в тестовом 
# массиве признаков медианой соответствующей переменной, 
# вычисленной по ОБУЧАЮЩЕМУ массиву признаков
test[numeric_cols] = simp.transform(test[numeric_cols])

# создаем список категориальных переменных
cat_cols = ['billtype', 'pay']
# создаем экземпляр класса SimpleImputer
simp2 = SimpleImputer(strategy='most_frequent')
# обучаем модель импутации - вычисляем моды переменных
# billtype и pay в обучающем массиве признаков
simp2.fit(train[cat_cols])
# применяем модель импутации к указанным переменным 
# в обучающем массиве признаков: 
# заменяем пропуски каждой переменной в обучающем 
# массиве признаков модой соответствующей переменной, 
# вычисленной по ОБУЧАЮЩЕМУ массиву признаков
train[cat_cols] = simp2.transform(train[cat_cols])
# применяем модель импутации к указанным переменным 
# в тестовом массиве признаков: 
# заменяем пропуски каждой переменной в тестовом 
# массиве признаков модой соответствующей переменной, 
# вычисленной по ОБУЧАЮЩЕМУ массиву признаков
test[cat_cols] = simp2.transform(test[cat_cols])

In [14]:
# взглянем на обучающий массив признаков
train.head(10)

,longdist,internat,local,age,income,billtype,pay,miss_ind_local
18,19.0,0.0,11.0,88.0,66906.60,Бесплатный,CH,False
19,19.0,0.0,96.0,79.0,37571.10,Бесплатный,CC,False
66,0.0,0.0,1.0,94.0,13946.80,Бюджетный,CC,False
65,9.0,6.0,12.0,93.0,46771.20,Бюджетный,CC,False
74,26.0,0.0,32.5,32.0,41031.40,Бюджетный,Auto,True
96,9.0,0.0,32.5,25.0,52366.40,Бюджетный,CC,True
47,14.0,0.0,48.0,58.0,4123.46,Бесплатный,CC,False
107,0.0,0.0,3.0,68.0,14955.80,Бесплатный,CH,False
11,1.0,0.0,9.0,44.0,11861.40,Бюджетный,CD,False
86,17.0,4.0,13.0,54.0,27376.00,Бюджетный,CC,False


In [15]:
# импортируем класс OneHotEncoder
from sklearn.preprocessing import OneHotEncoder
# создаем экземпляр класса OneHotEncoder
ohe = OneHotEncoder(sparse=False, handle_unknown='ignore')
# копируем переменную pay в обучающем массиве признаков
ohe_train = train[['pay']].copy()
# обучаем модель дамми-кодирования - определяем дамми для переменной
# pay в обучающем массиве признаков
ohe.fit(ohe_train)
# выполняем дамми-кодирование переменной 
# pay в обучающем массиве признаков
ohe_train_transformed = ohe.transform(ohe_train)
# смотрим первые 10 наблюдений
ohe_train_transformed[:10]

array([[0., 0., 0., 1.],
       [0., 1., 0., 0.],
       [0., 1., 0., 0.],
       [0., 1., 0., 0.],
       [1., 0., 0., 0.],
       [0., 1., 0., 0.],
       [0., 1., 0., 0.],
       [0., 0., 0., 1.],
       [0., 0., 1., 0.],
       [0., 1., 0., 0.]])

In [16]:
# копируем переменную pay в тестовом массиве признаков
ohe_test = test[['pay']].copy()
# заменяем значение в переменной HouseStyle
ohe_test.iloc[0, 0] = 'new_category'
# выводим первые три наблюдения
ohe_test.head(3)

,pay
44,new_category
59,CH
51,CC


In [17]:
# выполняем дамми-кодирование переменной 
# pay в тестовом массиве признаков
ohe_test_transformed = ohe.transform(ohe_test)
# смотрим первые три наблюдения
ohe_test_transformed[:3]

array([[0., 0., 0., 0.],
       [0., 0., 0., 1.],
       [0., 1., 0., 0.]])

In [18]:
# создаем обучающий датафрейм c одним столбцом
train = pd.DataFrame(
    {'City': ['MSK', 'MSK', 'MSK', 'SPB', 
              'EKB', 'EKB', 'EKB', 
              'EKB', 'EKB']})
train

,City
0,MSK
1,MSK
2,MSK
3,SPB
4,EKB
5,EKB
6,EKB
7,EKB
8,EKB


In [19]:
# смотрим частоты категорий City
train['City'].value_counts()

EKB    5
MSK    3
SPB    1
Name: City, dtype: int64

In [20]:
# создаем класс OneHotEncoder, задав порог 
# для редких категорий, и обучаем
ohe = OneHotEncoder(
    min_frequency=3,
    sparse=False,
    handle_unknown='infrequent_if_exist')
ohe.fit(train)
ohe.infrequent_categories_

[array(['SPB'], dtype=object)]

In [21]:
# выполняем дамми-кодирование 
# в обучающем датафрейме
ohe.transform(train)

array([[0., 1., 0.],
       [0., 1., 0.],
       [0., 1., 0.],
       [0., 0., 1.],
       [1., 0., 0.],
       [1., 0., 0.],
       [1., 0., 0.],
       [1., 0., 0.],
       [1., 0., 0.]])

In [22]:
# создаем обучающий датафрейм c одним столбцом,
# в котором будет новая категория 'NSK'
test = pd.DataFrame(
    {'City': ['NSK', 'MSK', 'NSK', 'MSK', 
              'SPB', 'EKB', 'SPB', 
              'EKB', 'SPB']})
test

,City
0,NSK
1,MSK
2,NSK
3,MSK
4,SPB
5,EKB
6,SPB
7,EKB
8,SPB


In [23]:
# выполняем дамми-кодирование 
# в тестовом датафрейме
ohe.transform(test)

array([[0., 0., 1.],
       [0., 1., 0.],
       [0., 0., 1.],
       [0., 1., 0.],
       [0., 0., 1.],
       [1., 0., 0.],
       [0., 0., 1.],
       [1., 0., 0.],
       [0., 0., 1.]])

In [24]:
# применяем функцию get_dummies()
train = pd.get_dummies(train)
train.head()

,City_EKB,City_MSK,City_SPB
0,0,1,0
1,0,1,0
2,0,1,0
3,0,0,1
4,1,0,0


In [25]:
# импортируем функцию csr_matrix()
from scipy.sparse import csr_matrix

# создаем массив из 16 элементов, 4 строки и 4 столбца
A = np.array([[0, 0, 0, 0], 
              [5, 8, 0, 0], 
              [0, 0, 3, 0], 
              [0, 6, 0, 0]])
# печатаем массив
print(A)
# преобразовываем в CRS-представление
print('')
S = csr_matrix(A)
# печатаем CRS-представление
print(S)
# преобразовываем обратно в плотный массив
D = S.todense()
# печатаем плотный массив
print('')
print(D)

[[0 0 0 0]
 [5 8 0 0]
 [0 0 3 0]
 [0 6 0 0]]

  (1, 0)	5
  (1, 1)	8
  (2, 2)	3
  (3, 1)	6

[[0 0 0 0]
 [5 8 0 0]
 [0 0 3 0]
 [0 6 0 0]]


In [26]:
# загружаем и смотрим данные
data = pd.read_csv('Data/catfeatures_challenge_II_train.csv')
data.head()

,id,bin_0,bin_1,bin_2,bin_3,bin_4,nom_0,nom_1,nom_2,nom_3,...,nom_9,ord_0,ord_1,ord_2,ord_3,ord_4,ord_5,day,month,target
0,0,0.0,0.0,0.0,F,N,Red,Trapezoid,Hamster,Russia,...,02e7c8990,3.0,Contributor,Hot,c,U,Pw,6.0,3.0,0
1,1,1.0,1.0,0.0,F,Y,Red,Star,Axolotl,NaN,...,f37df64af,3.0,Grandmaster,Warm,e,X,pE,7.0,7.0,0
2,2,0.0,1.0,0.0,F,N,Red,NaN,Hamster,Canada,...,NaN,3.0,NaN,Freezing,n,P,eN,5.0,9.0,0
3,3,NaN,0.0,0.0,F,N,Red,Circle,Hamster,Finland,...,f9d456e57,1.0,Novice,Lava Hot,a,C,NaN,3.0,3.0,0
4,4,0.0,NaN,0.0,T,N,Red,Triangle,Hamster,Costa Rica,...,c5361037c,3.0,Grandmaster,Cold,h,C,OZ,5.0,12.0,0


In [27]:
# избавляемся от редких категорий
for col in ['nom_5', 'nom_6', 'nom_7', 'nom_8', 'nom_9']:
    abs_freq = data[col].value_counts(dropna=False)
    data[col] = np.where(
        data[col].isin(abs_freq[abs_freq >= 100].index.tolist()), 
        data[col], 'Other')

In [28]:
# удаляем идентификатор
data.drop('id', axis=1, inplace=True)
# формируем массив меток и массив признаков
labels = data.pop('target').values
# смотрим типы переменных и количество пропусков
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 600000 entries, 0 to 599999
Data columns (total 23 columns):
 #   Column  Non-Null Count   Dtype  
---  ------  --------------   -----  
 0   bin_0   582106 non-null  float64
 1   bin_1   581997 non-null  float64
 2   bin_2   582070 non-null  float64
 3   bin_3   581986 non-null  object 
 4   bin_4   581953 non-null  object 
 5   nom_0   581748 non-null  object 
 6   nom_1   581844 non-null  object 
 7   nom_2   581965 non-null  object 
 8   nom_3   581879 non-null  object 
 9   nom_4   581965 non-null  object 
 10  nom_5   582222 non-null  object 
 11  nom_6   581869 non-null  object 
 12  nom_7   581997 non-null  object 
 13  nom_8   582245 non-null  object 
 14  nom_9   581927 non-null  object 
 15  ord_0   581712 non-null  float64
 16  ord_1   581959 non-null  object 
 17  ord_2   581925 non-null  object 
 18  ord_3   582084 non-null  object 
 19  ord_4   582070 non-null  object 
 20  ord_5   582287 non-null  object 
 21  day     58

In [29]:
# для всех столбцов создаем индикаторы пропусков
for col in data.columns:
    data[col + '_isnan'] = np.where(data[col].isnull(), 'T', 'F')

In [30]:
# создаем две новые переменные на основе переменной ord5,
# просто извлекая первый и второй символы
data['ord_5a'] = data['ord_5'].str[0]
data['ord_5b'] = data['ord_5'].str[1]

In [31]:
# формируем список столбцов
columns = [col for col in data.columns]

In [32]:
# разбиваем набор на обучающую и тестовую выборки
X_train, X_test, y_train, y_test = train_test_split(
    data, 
    labels, 
    test_size=0.3,
    stratify=labels,
    random_state=42)

In [33]:
# создаем дамми-переменные, не используем
# параметр sparse
X_tr_non_sparse = pd.get_dummies(
    X_train,
    columns=columns, 
    drop_first=True, 
    sparse=False)
X_tst_non_sparse = pd.get_dummies(
    X_test, 
    columns=columns, 
    drop_first=True, 
    sparse=False)

# создаем дамми-переменные, используем
# параметр sparse
X_tr_sparse = pd.get_dummies(
    X_train,
    columns=columns, 
    drop_first=True, 
    sparse=True)
X_tst_sparse = pd.get_dummies(
    X_test, 
    columns=columns, 
    drop_first=True, 
    sparse=True)

In [34]:
# смотрим формы массивов
print('non_sparse:', X_tr_non_sparse.shape, X_tst_non_sparse.shape)
print('sparse:', X_tr_sparse.shape, X_tst_sparse.shape)

non_sparse: (420000, 5026) (180000, 5026)
sparse: (420000, 5026) (180000, 5026)


In [35]:
# импортируем функцию roc_auc_score() для
# вычисления AUC-ROC
from sklearn.metrics import roc_auc_score
# импортируем класс LogisticRegression
from sklearn.linear_model import LogisticRegression

In [36]:
%%time

# обучаем модель логистической регрессии
# на массиве признаков, созданного с
# с помощью параметра sparse
logreg = LogisticRegression(solver='liblinear').fit(
    X_tr_sparse, y_train)
print("AUC на обучающей выборке: {:.3f}".format(
    roc_auc_score(y_train, logreg.predict_proba(
        X_tr_sparse)[:, 1])))
print("AUC на тестовой выборке: {:.3f}".format(
    roc_auc_score(y_test, logreg.predict_proba(
        X_tst_sparse)[:, 1])))

AUC на обучающей выборке: 0.801
AUC на тестовой выборке: 0.787
CPU times: user 30.4 s, sys: 520 ms, total: 30.9 s
Wall time: 11.4 s


In [37]:
%%time

# обучаем модель логистической регрессии
# на массиве признаков, созданного
# без помощи параметра sparse
logreg = LogisticRegression(solver='liblinear').fit(
    X_tr_non_sparse, y_train)
print("AUC на обучающей выборке: {:.3f}".format(
    roc_auc_score(y_train, logreg.predict_proba(
        X_tr_non_sparse)[:, 1])))
print("AUC на тестовой выборке: {:.3f}".format(
    roc_auc_score(y_test, logreg.predict_proba(
        X_tst_non_sparse)[:, 1])))

AUC на обучающей выборке: 0.801
AUC на тестовой выборке: 0.787
CPU times: user 1min 58s, sys: 29.2 s, total: 2min 27s
Wall time: 2min 2s
